In [ ]:
%%R
options(htmltools.dir.version = FALSE)
knitr::opts_chunk$set(
  message = FALSE,
  warning = FALSE,
  dev = "svg",
  fig.align = "center",
  #fig.width = 11,
  #fig.height = 5
  cache = FALSE
)

# define vars
om = par("mar")
lowtop = c(om[1],om[2],0.1,om[4])
library(tidyverse)
library(knitr)
library(reticulate)
use_python("C:\\Users\\jbpost2\\AppData\\Local\\Programs\\Python\\Python310\\python.exe")
#use_python("C:\\python\\python.exe")
options(dplyr.print_min = 5)
options(reticulate.repl.quiet = TRUE)

layout: false
class: title-slide-section-red, middle

# LASSO Models
Justin Post 

---
layout: true

<div class="my-footer"><img src="img/logo.png" style="height: 60px;"/></div> 

---

# Recap

- Judge the model's effectiveness using a **Loss** function

- Often split data into a training and test set
    + Perhaps 70/30 or 80/20
    
- Cross-validation gives a way to use more of the data while still seeing how the model does on test data
    - Commonly 5 fold or 10 fold is done
    - Once a best model is chosen, model is refit on entire data set

---

# Recap

- Judge the model's effectiveness using a **Loss** function

- Often split data into a training and test set
    + Perhaps 70/30 or 80/20
    
- Cross-validation gives a way to use more of the data while still seeing how the model does on test data
    - Commonly 5 fold or 10 fold is done
    - Once a best model is chosen, model is refit on entire data set

- Often use both! Let's see why by introducing a model with a **tuning parameter**

---

# LASSO Model

- [Least Angle Subset and Selection Operator](https://www.jstor.org/stable/2346178) or LASSO
    + Similar to Least Squares but a penalty is placed on the sum of the absolute values of the regression coefficients
    + $\alpha$ (>0) is called a tuning parameter
  
$$\min\limits_{\beta's}\sum_{i=1}^{n}(y_i-(\beta_0+\beta_1x_{1i}+...+\beta_px_{pi}))^2 + \alpha\sum_{j=1}^{p}|\beta_j|$$

---

# LASSO Model

- [Least Angle Subset and Selection Operator](https://www.jstor.org/stable/2346178) or LASSO
    + Similar to Least Squares but a penalty is placed on the sum of the absolute values of the regression coefficients
    + Sets coefficients to 0 as you 'shrink'!

In [ ]:
%%R
knitr::include_graphics("img/lasso_path.png")

---

# Tuning Parameter

- When choosing the tuning parameter, we are really considering a **family of models**!

In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model

bike_data = pd.read_csv("https://www4.stat.ncsu.edu/~online/datasets/bikeDetails.csv")
#create response and new predictor
bike_data['log_selling_price'] = np.log(bike_data['selling_price'])
bike_data['log_km_driven'] = np.log(bike_data['km_driven'])
lasso = linear_model.Lasso(alpha=0.1) 
lasso.fit(bike_data[["year", "log_km_driven"]].values, bike_data["log_selling_price"].values)

In [ ]:
from sklearn import linear_model
lasso = linear_model.Lasso(alpha=0.1) 
lasso.fit(bike_data[["year", "log_km_driven"]].values, bike_data["log_selling_price"].values)

In [ ]:
print(lasso.intercept_,lasso.coef_)

In [ ]:
lasso = linear_model.Lasso(alpha=1.05) 
lasso.fit(bike_data[["year", "log_km_driven"]].values, bike_data["log_selling_price"].values)

In [ ]:
lasso = linear_model.Lasso(alpha=1.05) 
lasso.fit(bike_data[["year", "log_km_driven"]].values, bike_data["log_selling_price"].values)

In [ ]:
print(lasso.intercept_,lasso.coef_)

---

# LASSO Fits Visual

- Perfect place for CV to help select the best $\alpha$!

In [ ]:
#code from https://www.kirenz.com/post/2019-08-12-python-lasso-regression-auto/
import matplotlib.pyplot as plt

alphas = np.linspace(0.01,2.2,500)
lasso = linear_model.Lasso(max_iter=10000)
coefs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(bike_data[["year", "log_km_driven"]].values, bike_data["log_selling_price"].values)
    coefs.append(lasso.coef_)

In [ ]:
ax = plt.gca()
ax.plot(alphas, coefs)
#ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('Standardized Coefficients')
plt.title('Lasso coefficients as a function of alpha');
plt.show()

---

# Using CV to Select the Tuning Parameter

- Return the optimal $\alpha$ using `LassoCV`

In [ ]:
from sklearn.linear_model import LassoCV
lasso_mod = LassoCV(cv=5, random_state=0, alphas = np.linspace(0,2.2,100)) \
    .fit(bike_data[["year", "log_km_driven"]].values,
          bike_data["log_selling_price"].values)

.left55[

In [ ]:
from sklearn.linear_model import LassoCV
lasso_mod = LassoCV(cv=5, random_state=0, alphas = np.linspace(0,2.2,100)) \
    .fit(bike_data[["year", "log_km_driven"]].values,
          bike_data["log_selling_price"].values)

In [ ]:
fit_info = np.array(list(zip(lasso_mod.alphas_, np.mean(lasso_mod.mse_path_, axis = 1))))
fit_info[fit_info[:,0].argsort()]

]


---

# Using CV to Select the Tuning Parameter

- Now fit using that optimal $\alpha$

In [ ]:
lasso_best = linear_model.Lasso(lasso_mod.alpha_) #warning thrown, but can ignore
lasso_best.fit(bike_data[["year", "log_km_driven"]].values, bike_data["log_selling_price"].values)
mlr_fit = linear_model.LinearRegression().fit(bike_data[["year", "log_km_driven"]].values, bike_data["log_selling_price"].values)

In [ ]:
lasso_best = linear_model.Lasso(lasso_mod.alpha_) #warning thrown, but can ignore
lasso_best.fit(bike_data[["year", "log_km_driven"]].values, bike_data["log_selling_price"].values)

In [ ]:
print(lasso_best.intercept_,lasso_best.coef_)

In [ ]:
#compare to MLR fit
mlr_fit = linear_model.LinearRegression() \
                      .fit(bike_data[["year", "log_km_driven"]].values, 
                           bike_data["log_selling_price"].values)

In [ ]:
print(mlr_fit.intercept_, mlr_fit.coef_)

---

# So Do We Just Need CV?

Sometimes!

- If you are only considering one type of model, you can use just a training/test set or k-fold CV

- When you have multiple types of models to choose from, usually use both!
    + When we use the test set too much, we may have '**data leakage**'
    + Essentially we end up training our models to the test set by using it too much
    
---

# Training/Validation/Test or CV/Test

- Instead, we sometimes split into a training, validation, and test set
- CV can be used to replace the validation set!

In [ ]:
%%R
knitr::include_graphics("img/training_validation_test.png")

---

# Training/Validation/Test or CV/Test

- Instead, we sometimes split into a training, validation, and test set
- CV can be used to replace the validation set!

In [ ]:
%%R
knitr::include_graphics("img/training_validation_test.png")

- Compare only the **best** model from each model type on the test set

---

# Recap

- LASSO is similar to an MLR model but shrinks coefficients and may set some to 0
    + Tuning parameter must be chosen (usually by CV)
    
- Training/Test split gives us a way to validate our model's performance
    - CV can be used on the training set to select **tuning parameters**
    - Helps determine the 'best' model for a class of models 
    
- With many competing model types, determine the best from each type check performance on the test set
